In [48]:
import load
import numpy as np
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import tree
import makegal

In [2]:
class Simplemock():
    def __init__(self, repo="/home/hoseung/Work/pyclusterevol/repo/sed/",
                 filter_system="SDSS",
                 sed_model="bc03",
                 load=True):
        self.filter_system = filter_system
        self.repo = repo
        self.sed_model = sed_model
        if load is True:
            self.load_filters()
            self.load_SED_wavelength()
            self.load_SED_all()

    def load_filters(self):        
        if self.filter_system == "SDSS":
            filter_lambda, filter_u, filter_g, filter_r, filter_i, filter_z = \
                np.genfromtxt(self.repo + "filter_sdss.dat",
                                skip_header=1, unpack=True)
        
            self.filters = {"lambda":filter_lambda,
                            "u":filter_u,
                            "g":filter_g,
                            "r":filter_r,
                            "i":filter_i,
                            "z":filter_z}    

    def load_SED_wavelength(self):
        if self.sed_model == "bc03":
            self.sed_wavelength = np.genfromtxt(self.repo + "lambda.dat")
            
    def load_SED_all(self):
        """
        Full SEDs are just a few tens of MB.
        """
        if self.sed_model == "bc03":
            self.metal_points = np.array([0.0004, 0.001, 0.004, 0.01, 0.02, 0.04])
            # age points in tables.
            self.age_points = np.genfromtxt(self.repo+"ages_yybc.dat") # in Gry unit
            self.SEDs = np.zeros((6, 221, 1221))
            
            for i, metal in enumerate(self.metal_points):
                self.SEDs[i,:,:] = np.genfromtxt(self.repo +
                                            "bc03_yy_{:.4f}".format(metal)).reshape(221, 1221)
        else:
            print("Sorry, Only bc03 is implemented.")
            
    def get_flux(self, star,
                 extinction = False,
                 metal_lower_cut = True,
                 filter_name='r'):
        ### star data ########################################################
        
        starmetal = star["metal"] # Is the original array modified?
        if metal_lower_cut:
            # No star with metallicity lower than the lowest table.
            starmetal[starmetal < min(self.metal_points)] = min(self.metal_points) * 1.0001 

        locate_metal = np.digitize(starmetal, self.metal_points)-1 # GOOD
        relevant_metals = self.metal_points[:max(locate_metal)+2]
        nmetals = len(relevant_metals)

        # Star Age
        t_univ = 13.7
        starage = t_univ - gg1.star["time"]

        locate_age = np.digitize(starage, self.age_points)-1 # GOOD
        relevant_ages = self.age_points[:max(locate_age)+2]
        nages = len(relevant_ages)
        
        ### Filter optimization. #################################################

        # Pick one
        this_filter = self.filters[filter_name]

        # band range
        i_filter_pos = this_filter > 0

        this_filter = this_filter[i_filter_pos]
        filter_lambda_this_band = self.filters["lambda"][i_filter_pos]

        lambda_min_this_band = min(filter_lambda_this_band)
        lambda_max_this_band = max(filter_lambda_this_band)

        i_lambda_min = np.argmax(self.sed_wavelength > lambda_min_this_band) -1
        #print(i_lambda_min, wavelength[i_lambda_min], lambda_min_this_band)
        i_lambda_max = np.argmax(self.sed_wavelength > lambda_max_this_band)
        #print(i_lambda_max, wavelength[i_lambda_max], lambda_max_this_band)

        # Only a small part of SED is needed.
        wavelength = self.sed_wavelength[i_lambda_min:i_lambda_max+1]
        n_wavelength = i_lambda_max - i_lambda_min + 1

        # Wavelengths at which filter function are defined are different from the SED wavelength points.
        # Interpolate filter function on SED points.
        filter_in_sed_wavelengths = np.interp(wavelength, filter_lambda_this_band, this_filter)
        
        ##### Caclulate band flux #################
        # Load only necessary data
        # Load all once, keep under the class and copy part of it when needed heere. 
        seds = np.zeros((nmetals, nages, n_wavelength)) # metal age lambda
        if self.sed_model == "bc03":
            for i, metal in enumerate(relevant_metals):
                seds[i,:,:] = self.SEDs[i,:nages, i_lambda_min:i_lambda_max+1]

        # All are array-wise calculations.
        # interpolation weight
        dl_m = (starmetal - relevant_metals[locate_metal] ) / \
                                              (relevant_metals[locate_metal+1] - relevant_metals[locate_metal])
        dr_m = (relevant_metals[locate_metal+1] - starmetal) / \
                                              (relevant_metals[locate_metal+1] - relevant_metals[locate_metal])
        dl_a = (starage - relevant_ages[locate_age] )   / \
                                              (relevant_ages[locate_age+1] - relevant_ages[locate_age])
        dr_a = (relevant_ages[locate_age+1] - starage ) / \
                                              (relevant_ages[locate_age+1] - relevant_ages[locate_age])


        # 2D linear interpolation
        # weight * SED.
        Flux =  np.multiply( (dr_m * dr_a), seds[locate_metal, locate_age,:].T).T +\
                np.multiply( (dl_m * dr_a), seds[locate_metal + 1, locate_age,:].T).T +\
                np.multiply( (dr_m * dl_a), seds[locate_metal, locate_age + 1, :].T).T +\
                np.multiply( (dl_m * dl_a), seds[locate_metal + 1, locate_age + 1,:].T).T

        # Convolve filter
        Flux = np.multiply(filter_in_sed_wavelengths, Flux)

        if not extinction:
            return np.sum(Flux, axis=1)
        else:
            print("Extinction - Not yet implemented")
            return

In [29]:
cd ~/Work/data/Horizon-AGN/

/media/hoseung/btrfs/Horizon-AGN


In [140]:
cd ~/Work/data/NH/

/home/hoseung/Work/data/NH


In [141]:
nout = 312#782 # 312
s = load.sim.Sim(nout=nout)
gcat = tree.halomodule.Halo(nout=nout, is_gal=True)

[sim._hilbert_cpulist] No AMR instance,
[sim._hilbert_cpulist] Loading one...
An AMR instance is created
 Sim
An AMR instance is created
 Sim
Simulation set up.
[Halo.load_info] loading info
[Halo.load_info] nout = 312, base =./


In [119]:
from matplotlib.colors import LogNorm

fig, axs = plt.subplots(1,2)
fig.set_size_inches(8,4)
for gdata in gcat.data[range(1004,2004,10)]:
    gg = load.rd_GM.Gal(nout, catalog=gdata.copy(), info=s.info)
    gg.debug=False
    #print(len(gg.star))
    is_good = makegal.mk_gal(gg, mstar_min=1e10, den_lim=1e7, verbose=False)
    #print(len(gg.star))
    #print("vz mean", gg.star["vz"].mean())
    gg.star["vx"] -= gg.star["vx"].mean()
    gg.star["vy"] -= gg.star["vy"].mean()
    gg.star["vz"] -= gg.star["vz"].mean()
    #print("vx mean", gg.star["vx"].mean())
    #print("vy mean", gg.star["vy"].mean())
    #print("vz mean", gg.star["vz"].mean())
    gg.reorient(pops=["star"])
    axs[0].hist2d(gg.star["y"], gg.star["x"], bins=200, norm=LogNorm())
    axs[1].hist2d(gg.star["z"], gg.star["y"], bins=200, norm=LogNorm())
    axs[0].set_aspect("equal")
    axs[0].set_aspect("equal")
    
    plt.savefig(str(gdata["id"]).zfill(5) + ".png")
    axs[0].cla()
    axs[1].cla()

plt.close()

info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001005
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001005
No CELL data loaded
catalog center 40.92744064331055 -0.11120238155126572 45.76774978637695
star x [ 40.949  40.909  40.933 ...,  40.931  40.928  40.934]
star x [ 21.271 -18.936   6.05  ...,   3.395   1.034   6.233]
gal.debug False
info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001015
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001015
No CELL data loaded
catalog center -35.69791793823242 31.10289192199707 -48.456119537353516
star x [-35.663 -35.672 -35.691 ..., -35.697 -35.697 -35.699]
star x [ 34.466  26.176   6.931 ...,   1.125   1.282  -1.03 ]
gal.debug False
info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001025
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001025
No CELL data load

info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001225
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001225
No CELL data loaded
catalog center -16.746509552001953 6.440572738647461 3.388202428817749
star x [-16.755 -16.747 -16.762 ..., -16.744 -16.756 -16.747]
star x [ -8.663  -0.938 -15.961 ...,   2.979  -9.928  -0.904]
gal.debug False
info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001235
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001235
No CELL data loaded
catalog center -52.97798538208008 56.72981643676758 -56.026092529296875
star x [-52.974 -52.967 -52.974 ..., -52.974 -52.977 -52.976]
star x [  4.112  11.169   3.757 ...,   4.047   1.282   1.881]
gal.debug False
info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001245
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001245
No CELL data loade

info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001435
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001435
No CELL data loaded
catalog center -46.09928894042969 22.07028579711914 -45.47291946411133
star x [-46.116 -46.102 -46.096 ..., -46.101 -46.099 -46.1  ]
star x [-16.319  -3.113   3.258 ...,  -1.64    0.172  -0.599]
gal.debug False
info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001445
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001445
No CELL data loaded
catalog center 29.187795639038086 12.366231918334961 -50.49340057373047
star x [ 29.199  29.187  29.184 ...,  29.192  29.187  29.188]
star x [ 11.669  -0.847  -3.658 ...,   4.175  -1.223   0.134]
gal.debug False
info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001455
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001455
No CELL data loade

info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001645
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001645
No CELL data loaded
catalog center -17.996076583862305 45.23554611206055 -49.75701141357422
star x [-17.997 -17.999 -17.996 ..., -17.997 -17.999 -17.995]
star x [-0.957 -3.241  0.477 ..., -0.954 -3.115  0.956]
gal.debug False
Not enough stars: 9.20e+09 Msun
2640 Aborting... 

info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001655
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001655
No CELL data loaded
catalog center 35.788169860839844 -42.5973014831543 -16.22087287902832
star x [ 35.788  35.787  35.81  ...,  35.785  35.79   35.785]
star x [ -0.58   -1.152  21.675 ...,  -3.307   2.083  -3.399]
gal.debug False
info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001665
No DM data loaded
File Not Found: ./GalaxyMaker/CE

File Not Found: ./halo/HAL_00782/hal_dms_0001855
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001855
No CELL data loaded
catalog center 30.949018478393555 7.430619716644287 8.486104011535645
star x [ 30.954  30.953  30.953 ...,  30.95   30.954  30.95 ]
star x [ 5.14   3.853  3.677 ...,  1.39   5.133  0.799]
gal.debug False
info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001865
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001865
No CELL data loaded
catalog center -36.284912109375 -67.57096862792969 65.94866180419922
star x [-36.261 -36.286 -36.285 ..., -36.351 -36.336 -36.3  ]
star x [  2.354e+01  -1.041e+00  -2.289e-02 ...,  -6.610e+01  -5.064e+01
  -1.525e+01]
gal.debug False
info <load.info.Info object at 0x7fc0141fddd8>
File Not Found: ./halo/HAL_00782/hal_dms_0001875
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0001875
No CELL data loaded
catalog center 19.023288726806

In [145]:
s = load.sim.Sim(nout=nout)

gg1 = load.rd_GM.Gal(nout, catalog=gcat.data[21].copy(), info=s.info)
gg1.debug=False
make_gal.mk_gal(gg1)
print(np.log10(gg1.meta.mstar))
#gg1.cal_norm_vec()
#gg1.meta.nvec

[sim._hilbert_cpulist] No AMR instance,
[sim._hilbert_cpulist] Loading one...
An AMR instance is created
 Sim
An AMR instance is created
 Sim
Simulation set up.
info <load.info.Info object at 0x7fc00f9db240>
File Not Found: ./halo/HAL_00312/hal_dms_0000022
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00312/gal_cells_0000022
No CELL data loaded
gal.debug False
10.3975496195


In [6]:
MockSED = Simplemock()

In [7]:
gg1.star["metal"][gg1.star["metal"] >= 0.04] = 0.04 - 1e-9
## Improve Boundary cases. !!



In [121]:
plt.hist(gg1.star["metal"])
plt.show()

In [122]:
sum(gg1.star["metal"] > 0.04)

0

In [224]:
gg1.star.shape

(2696173,)

In [131]:
Flux_u = MockSED.get_flux(star=gg1.star, filter_name='u')
Flux_g = MockSED.get_flux(star=gg1.star, filter_name='g')
Flux_r = MockSED.get_flux(star=gg1.star, filter_name='r')
Flux_i = MockSED.get_flux(star=gg1.star, filter_name='i')
Flux_z = MockSED.get_flux(star=gg1.star, filter_name='z')

In [132]:
# Can rotate galaxy. 
gg1.reorient(pops=["star"])

In [124]:
plt.hist(Flux_g)
plt.show()

In [11]:
max(Flux_i)

1.539564509816072

In [12]:
min(Flux_i)

0.15811099005896428

In [139]:
np.log10(gg1.meta.mstar)

11.193673355473939

In [138]:
fig, axs = plt.subplots(2,3)

x1 = gg1.star["x"]
x2 = gg1.star["y"]

npix = 200
from matplotlib.colors import LogNorm

titles=["u","g","r","i","z","composite"]
for i, ax in enumerate(axs.ravel()[:5]):
    flux = [Flux_u, Flux_g, Flux_r, Flux_i, Flux_z][i]
    ax.hist2d(x1,x2,
           weights=np.log10(flux+1),
           bins=npix,
           cmap=plt.cm.binary_r,
           norm=LogNorm(),
           range=[[-30,30]]*2)
    ax.set_aspect("equal")
    ax.set_title(titles[i])
    
    #labels = [item.get_text() for item in ax.get_xticklabels()]
    #empty_string_labels = ['']*len(labels)
    #ax.set_xticklabels(empty_string_labels)
    #ax.set_yticklabels(empty_string_labels)


# Try scaling Flux_x arrays to make a better composite image   
comp_img = composite_rgb(x1,x2,
                         Flux_i,
                         Flux_r,
                         Flux_g,
                         npix=npix,
                         multiply_r = 1.3,
                         multiply_g = 1.0,
                         multiply_b = 1.3,
                         log_scale=True,
                         range=[[-30,30]]*2)
ax = axs[-1][-1]
ax.imshow(comp_img)
ax.set_title("composite")
labels = [item.get_text() for item in ax.get_xticklabels()]
empty_string_labels = ['']*len(labels)
ax.set_aspect("equal")
ax.set_xticklabels(empty_string_labels)
ax.set_yticklabels(empty_string_labels)

plt.show()

In [135]:
def composite_rgb(x,y, weight_r, weight_g, weight_b,
                  npix=100,
                  multiply_r=1.3,
                  multiply_g=1.1,
                  multiply_b=1.0,
                  log_scale = True,
                  range=None):
    from PIL import Image

    rgbArray = np.zeros((npix,npix,3))

    rgbArray[..., 0] = np.histogram2d(x, y,
               weights=weight_r, bins=npix, range=range)[0] * multiply_r
    rgbArray[..., 1] = np.histogram2d(x,y,
               weights=weight_g, bins=npix, range=range)[0] * multiply_g
    rgbArray[..., 2] = np.histogram2d(x,y,
               weights=weight_b, bins=npix, range=range)[0] * multiply_b
    
    if log_scale:
        rgbArray = np.log10(rgbArray+1)
    rgbArray = rgbArray/rgbArray.max() * 255
    #return rgbArray
    
    img = Image.fromarray(rgbArray.astype('uint8'))
    
    return img.rotate(90)
    #img.save('myimg.jpeg')